<a href="https://colab.research.google.com/github/yyujnn/Deep_Learning/blob/main/ex01_%EB%94%A5%EB%9F%AC%EB%8B%9D%EB%A7%9B%EB%B3%B4%EA%B8%B0(%EC%84%B1%EC%A0%81%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%98%88%EC%B8%A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 실행
# shift + enter : 현재 셀을 실행하고 아래로 커서 이동(아래로 셀이 없다면 생성하고 커서 이동)
# ctrl + enter : 현재 셀 실행

# 셀 생성
# 위에 셀 생성 : ctrl + m + a
# 아래에 셀 생성 : ctrl + m + b

# 마크다운 : ctrl + m + m(연속적으로 누르고 있으면 자동 마크다운 변환)
# 원래 상태로 전환 : ctrl + m + y

# 셀 삭제 : ctrl + m + d
# 하나의 셀 안에 있는 동일한 단어 한꺼번에 선택 : ctrl + shift + l

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# /content/drive/MyDrive/Colab Notebooks/DeepLearning_빅데/data/student-mat.csv
# 현재 경로
#%pwd
# 경로이동
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning_빅데

/content/drive/MyDrive/Colab Notebooks/DeepLearning_빅데


### 목표
- 학생 수학 성적 예측하는 회귀모델을 만들어보자!
- keras를 이용하여 딥러닝 신경망을 구성해보자!

In [ ]:
# 데이터 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# csv, 구분자 : ';'
data = pd.read_csv('./data/student-mat.csv', delimiter= ';')
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [ ]:
# 컬럼명 출력
data.columns

# 답 데이터 : G3(수학성적)
# 특성 데이터 : studytime(공부시간), 특성 1개 학습 진행

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [ ]:
data['G3'] # 연속적인 수치값 성적 -> 회귀

0       6
1       6
2      10
3      15
4      10
       ..
390     9
391    16
392     7
393    10
394     9
Name: G3, Length: 395, dtype: int64

In [ ]:
# 문제와 답으로 분리
X = data['studytime'] # 공부시간 1차원 배열
y = data['G3']  # G3 1차원 배열

# train, test 분리 7:3, 3
# sklearn 도구 활용
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 3)

# 크기 확인
print('훈련용:', X_train.shape, y_train.shape) # 특성 1개
print('테스트용:', X_test.shape, y_test.shape)

훈련용: (276,) (276,)
테스트용: (119,) (119,)


#### 신경망 모델 만들기
- 1. 신경망 구조 설계 : 뼈대 구축, 층 쌓기
- 2. 신경망 학습/ 평가 방법 설정
- 3. 신경망 모델 학습 및 결과 시각화
- 4. 모델 평가 및 예측

In [ ]:
# 1. 신경망 구조 설계
from tensorflow.keras import Sequential # 뼈대 구축 기능
from tensorflow.keras.layers import Dense, Activation # 층 쌓기

In [ ]:
# 모델 정의
model = Sequential() # 뼈대 구축

# 층 쌓기 시작!
# 입력층(데이터 설명 => 특성 개수 1) + 중간층1(뉴런 or 유닛 개수 설정, activation)
# input_dim = 우리가 사용할 특성 개수 (= studytime)
model.add(Dense(units = 4, input_dim = 1))
model.add(Activation('sigmoid')) # 활성화 함수 : 반응 결과 정리하는 기능

# 출력층 -- "회귀"일땐 units = 1 (고정), 활성화 함수 activation = 'linear'  (항등함수, 생략이 가능하다)
# 선형함수 -> 뉴런 -> 유닛
model.add(Dense(units = 1))
# model.add(Activation('linear')) => 생략이 가능

# 요약
model.summary()

In [ ]:
# 2. 신경망 학습/ 평가 방법 설정
# compile()
# 모델을 설계를 pc가 이해할 수 있도록 번역하는 과정
# 오차 계산 공식, 최적화 함수, 평가 지표(척도)
# 회귀
model.compile(loss = 'mse', # 손실함수(비용함수) : mse(평균제곱오차) -> 회귀
              optimizer = 'sgd', # 최적화함수 : 경사하강법 종류 설정 -> 확률적 경사 하강법
              ) # metrics 키워드 사용 'mse' 활용

In [ ]:
# 3. 학습
# h 라는 변수에 학습 현황 저장
h = model.fit(X_train, y_train, epochs=200)
h

# h안에 있는 학습 히스토리에 접근
h.history['loss']
# h.history['history']

Epoch 1/200
9/9 [==============================] - 1s 4ms/step - loss: 72.3114
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 32.3085
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 23.0387
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 21.2001
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 20.8484
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 20.8186
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 20.7961
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 20.7795
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 20.7983
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 20.8657
Epoch 11/200
9/9 [==============================] - 0s 2ms/step - loss: 20.8021
Epoch 12/200
9/9 [==============================] - 0s 2ms/step - loss: 20.7850
Epoch 13/200
9/9 [==============================]

In [ ]:
# 모델 평가
# score -> evaluate()
# 오차 mse
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 4ms/step - loss: 21.0733


21.073266983032227

In [ ]:
# 예측 predict
model.predict(X_test)

4/4 [==============================] - 0s 3ms/step


array([[10.425052],
       [10.83905 ],
       [10.83905 ],
       [10.425052],
       [10.83905 ],
       [ 9.784534],
       [10.425052],
       [11.108976],
       [ 9.784534],
       [ 9.784534],
       [ 9.784534],
       [10.425052],
       [10.425052],
       [10.83905 ],
       [11.108976],
       [10.83905 ],
       [10.83905 ],
       [10.425052],
       [10.425052],
       [10.83905 ],
       [10.425052],
       [10.425052],
       [ 9.784534],
       [10.425052],
       [ 9.784534],
       [10.425052],
       [10.425052],
       [ 9.784534],
       [10.425052],
       [10.83905 ],
       [11.108976],
       [ 9.784534],
       [10.83905 ],
       [ 9.784534],
       [10.83905 ],
       [10.425052],
       [10.425052],
       [ 9.784534],
       [10.425052],
       [10.83905 ],
       [10.425052],
       [10.425052],
       [10.425052],
       [10.425052],
       [10.425052],
       [11.108976],
       [10.83905 ],
       [ 9.784534],
       [ 9.784534],
       [ 9.784534],
